In [21]:
import org.apache.spark.sql.functions._
import org.apache.spark.sql.Row
import org.apache.hadoop.fs.{FileSystem, Path}
import org.apache.spark.rdd.RDD
import scala.util.matching.Regex
import org.apache.spark.sql.types._

In [22]:
val spark = SparkSession.builder()
      .appName("CaseStudy3 - Handling Incomplete Metadata")
      .getOrCreate()

spark = org.apache.spark.sql.SparkSession@42ddb1a5


org.apache.spark.sql.SparkSession@42ddb1a5

In [23]:
val moviesDataPath = "gs://spark-tasks-bucket/day_16_17/movie.csv"
val moviesDF = spark.read.option("header", "true").option("inferSchema", "true").csv(moviesDataPath)

moviesDataPath = gs://spark-tasks-bucket/day_16_17/movie.csv
moviesDF = [movieId: int, title: string ... 1 more field]


[movieId: int, title: string ... 1 more field]

In [24]:
val moviesRDD = moviesDF.rdd.map {row =>
    val movieId = row.getAs[Int]("movieId")
    val title = row.getAs[String]("title")
    val genres = row.getAs[String]("genres")
    
    (movieId, title, genres)
}

moviesRDD = MapPartitionsRDD[15] at map at <console>:44


MapPartitionsRDD[15] at map at <console>:44

In [25]:
val standardReleaseYear = 9999

standardReleaseYear = 9999


9999

In [26]:
// Extract the release years and save it as metadata.json in gs

def extractYear(title: String): Int = {
    val lastIndexOfOpenParen = title.lastIndexOf("(")
    val lastIndexOfCloseParen = title.lastIndexOf(")")
    
    if (lastIndexOfOpenParen != -1 && lastIndexOfCloseParen != -1) {
        val yearCandidate = title.substring(lastIndexOfOpenParen + 1, lastIndexOfCloseParen)
        if (yearCandidate.forall(_.isDigit) && yearCandidate.length == 4) yearCandidate.toInt 
        else standardReleaseYear
    } else standardReleaseYear
}

val metadataRDD: RDD[(Integer, Integer)] = moviesRDD.map { case (movieId, title, genres) =>
  val year = extractYear(title)
  (movieId, year)
}

metadataRDD = MapPartitionsRDD[16] at map at <console>:59


extractYear: (title: String)Int


MapPartitionsRDD[16] at map at <console>:59

In [27]:
val metadataDF = metadataRDD.toDF("movieId", "releaseYear")

metadataDF = [movieId: int, releaseYear: int]


[movieId: int, releaseYear: int]

In [30]:
metadataDF.filter(col("releaseYear") === 9999).show()

+-------+-----------+
|movieId|releaseYear|
+-------+-----------+
|  40697|       9999|
|  79607|       9999|
|  87442|       9999|
|  98063|       9999|
| 107434|       9999|
| 108548|       9999|
| 108583|       9999|
| 112406|       9999|
| 113190|       9999|
| 115133|       9999|
| 115685|       9999|
| 125571|       9999|
| 125632|       9999|
| 125958|       9999|
| 126438|       9999|
| 126929|       9999|
| 127005|       9999|
| 128612|       9999|
| 128734|       9999|
| 129651|       9999|
+-------+-----------+
only showing top 20 rows



In [31]:
metadataDF.coalesce(1).write.mode("overwrite").json("gs://spark-tasks-bucket/day_16_17/metadata")

In [32]:
// Read the metadata json

val metadataJsonDF = spark.read.json("gs://spark-tasks-bucket/day_16_17/metadata")

metadataJsonDF = [movieId: bigint, releaseYear: bigint]


[movieId: bigint, releaseYear: bigint]

In [36]:
metadataJsonDF.filter(col("releaseYear").isNull).show()

+-------+-----------+
|movieId|releaseYear|
+-------+-----------+
+-------+-----------+



In [37]:
val metadataJsonRDD = metadataJsonDF.rdd.map { row =>
    val movieId = row.getAs[Long]("movieId").toInt
    val releaseYear = row.getAs[Long]("releaseYear").toInt
    
    (movieId, releaseYear)
}

metadataJsonRDD = MapPartitionsRDD[42] at map at <console>:44


MapPartitionsRDD[42] at map at <console>:44

In [39]:
// Join

val joinedRDD = moviesRDD.map{ case (movieId, title, genres) => (movieId, (title, genres)) }
                         .join(metadataJsonRDD)

joinedRDD = MapPartitionsRDD[50] at join at <console>:48


MapPartitionsRDD[50] at join at <console>:48

In [45]:
val joinedDF = joinedRDD.map { case(movieId, ((title, genre), release)) =>
    if(release != 9999) (movieId, title, genre, release)
    else (movieId, s"$title (9999)", genre, release)
}.toDF("movieId", "title", "genre", "releaseYear")

joinedDF = [movieId: int, title: string ... 2 more fields]


[movieId: int, title: string ... 2 more fields]

In [46]:
joinedDF.filter(col("releaseYear").isNull).show()

+-------+-----+-----+-----------+
|movieId|title|genre|releaseYear|
+-------+-----+-----+-----------+
+-------+-----+-----+-----------+



In [47]:
val output_path = "hdfs:///user/day_16_17/case_study_3"
joinedDF.write.mode("overwrite").parquet(output_path)

output_path = hdfs:///user/day_16_17/case_study_3


hdfs:///user/day_16_17/case_study_3

In [48]:
// Try reading this data
spark.read.parquet(output_path).show(20)

+-------+--------------------+--------------------+-----------+
|movieId|               title|               genre|releaseYear|
+-------+--------------------+--------------------+-----------+
| 113843|Killing Us Softly...|         Documentary|       2010|
| 103301|   Liz & Dick (2012)|               Drama|       2012|
| 110163|  Aujourd'hui (2012)|               Drama|       2012|
|  91902|        Elena (2011)|               Drama|       2011|
|  68522|        Earth (2007)|         Documentary|       2007|
| 111517|10.000 Km (Long D...|       Drama|Romance|       2014|
| 100306|       Angst  (1983)|        Drama|Horror|       1983|
|   5354|Cactus Flower (1969)|              Comedy|       1969|
|   4926|Everybody's Famou...|Comedy|Drama|Musical|       2000|
| 100494| Incir Reçeli (2011)|       Drama|Romance|       2011|
| 117509| City Slacker (2012)|             Romance|       2012|
|   4992|Kate & Leopold (2...|      Comedy|Romance|       2001|
|  92477|Yes: 9012 Live (1...| Documenta

In [49]:
spark.read.parquet(output_path).filter(col("releaseYear") === 9999).show(20)

+-------+--------------------+--------------------+-----------+
|movieId|               title|               genre|releaseYear|
+-------+--------------------+--------------------+-----------+
| 126929|Li'l Quinquin (9999)|  (no genres listed)|       9999|
| 126438|Two: The Story of...|   Documentary|Drama|       9999|
| 125571|The Court-Martial...|  (no genres listed)|       9999|
| 112406|Brazil: In the Sh...|         Documentary|       9999|
|  98063|Mona and the Time...|               Drama|       9999|
|  40697|    Babylon 5 (9999)|              Sci-Fi|       9999|
| 128734|Polskie gówno (9999)|      Comedy|Musical|       9999|
| 113190|Slaying the Badge...|         Documentary|       9999|
| 128612|   Body/Cialo (9999)|Comedy|Drama|Mystery|       9999|
| 125958|Stephen Fry In Am...|  (no genres listed)|       9999|
| 125632|In Our Garden (9999)|  (no genres listed)|       9999|
| 129651|The Third Reich: ...|  (no genres listed)|       9999|
| 108583|Fawlty Towers (19...|          

In [50]:
spark.stop()

Waiting for a Spark session to start...